## 拓展数据集
### 与官方数据集噪音部分结合

In [13]:
import pandas as pd
import os
import pathlib
import glob
import matplotlib. pyplot as plt
import numpy as np
import seaborn as sns
import librosa
from pydub import AudioSegment
import random
import sklearn as sk 
import scipy
from sklearn import preprocessing
from sklearn import model_selection

In [14]:
data_dir = pathlib.Path("speech_commands_v0.02")
[x for x in data_dir.iterdir() if x.is_dir()]

[WindowsPath('speech_commands_v0.02/go'),
 WindowsPath('speech_commands_v0.02/max'),
 WindowsPath('speech_commands_v0.02/no'),
 WindowsPath('speech_commands_v0.02/off'),
 WindowsPath('speech_commands_v0.02/on'),
 WindowsPath('speech_commands_v0.02/stop'),
 WindowsPath('speech_commands_v0.02/wow'),
 WindowsPath('speech_commands_v0.02/_background_noise_')]

In [15]:
from os import listdir
from os.path import isfile, join, isdir
onlydirs = [d for d in listdir(data_dir) if isdir(join(data_dir, d))]
onlydirs

['go', 'max', 'no', 'off', 'on', 'stop', 'wow', '_background_noise_']

In [16]:
len(onlydirs)

8

In [17]:
our_wav = [f for f in os.listdir(join(data_dir, "max")) if f.endswith(".wav")]

In [18]:
our_wav

['max_100_central.wav',
 'max_100_left.wav',
 'max_100_right.wav',
 'max_101_central.wav',
 'max_101_left.wav',
 'max_101_right.wav',
 'max_102_central.wav',
 'max_102_left.wav',
 'max_102_right.wav',
 'max_103_central.wav',
 'max_103_left.wav',
 'max_103_right.wav',
 'max_104_central.wav',
 'max_104_left.wav',
 'max_104_right.wav',
 'max_105.wav',
 'max_106.wav',
 'max_107_central.wav',
 'max_107_left.wav',
 'max_107_right.wav',
 'max_108_central.wav',
 'max_108_left.wav',
 'max_108_right.wav',
 'max_109_central.wav',
 'max_109_left.wav',
 'max_109_right.wav',
 'max_10_central.wav',
 'max_10_left.wav',
 'max_10_right.wav',
 'max_110_central.wav',
 'max_110_left.wav',
 'max_110_right.wav',
 'max_11_central.wav',
 'max_11_left.wav',
 'max_11_right.wav',
 'max_12_central.wav',
 'max_12_left.wav',
 'max_12_right.wav',
 'max_13_central.wav',
 'max_13_left.wav',
 'max_13_right.wav',
 'max_14_central.wav',
 'max_14_left.wav',
 'max_14_right.wav',
 'max_15_central.wav',
 'max_15_left.wav',
 '

In [19]:
our_audiosegment = []
for wav in our_wav:
    samples = AudioSegment.from_wav(join(join(data_dir, "max"), wav))
    our_audiosegment.append(samples)

In [20]:
our_audiosegment

In [21]:
background = [f for f in os.listdir(join(data_dir, "_background_noise_")) if f.endswith(".wav")]

In [22]:
background

['doing_the_dishes.wav',
 'dude_miaowing.wav',
 'exercise_bike.wav',
 'pink_noise.wav',
 'running_tap.wav',
 'white_noise.wav']

In [23]:
background_noise = []
for wav in background:
    samples = AudioSegment.from_wav(join(join(data_dir, "_background_noise_"), wav))
    background_noise.append(samples)

In [24]:
background_noise

In [25]:
SAMPLE_NUM = 3 # 每个数据随机生成3条

for our_num, _ in enumerate(our_audiosegment):
    for noise_num, _ in enumerate(background_noise):
        sound = our_audiosegment[our_num]
        for sn in range(SAMPLE_NUM):
            duration = len(our_audiosegment[our_num])
            selected_noise = background_noise[noise_num]
            start_idx = random.randint(0, len(selected_noise) - 1 - duration)
            noise = selected_noise[start_idx:(start_idx + duration)]
            # noise作为背景音的音量应为sound的一半
            noise = noise - ((1.5 * abs(sound.dBFS)) - abs(noise.dBFS))
            combined = sound.overlay(noise)
            combined.export(join(join(data_dir, "max"), f"sound{our_num}_noise{noise_num}_{sn}.wav"), format="wav")